In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 데이터 확인

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

In [ ]:
train

,index,누적전력량,유효전력평균,무효전력평균,주파수,전류평균,상전압평균,선간전압평균,온도,R상유효전력,R상무효전력,R상전류,R상전압,R상선간전압,S상유효전력,S상무효전력,S상전류,S상전압,S상선간전압,T상유효전력,T상무효전력,T상전류,T상전압,T상선간전압,label_역률평균,label_전류고조파평균,label_전압고조파평균
0,0,2248930.50,28963.0,20237.0,59.854076,45.197918,259.916656,449.916656,47.5000,8663.00,6182.00,40.84375,258.250,448.250,9883.00,6751.00,46.03125,260.750,452.750,10417.00,7304.00,48.71875,260.750,448.750,정상,경고,주의
1,1,0.00,0.0,0.0,0.000000,101.312500,0.000000,0.000000,24.3750,0.00,0.00,101.75000,0.000,0.000,0.00,0.00,101.50000,0.000,0.000,0.00,0.00,100.68750,0.000,0.000,경고,정상,정상
2,2,5375707.00,35244.0,19826.0,59.975650,107.385414,125.416664,216.750000,18.7500,11988.00,7384.00,111.53125,126.000,217.750,11020.00,6272.00,101.06250,125.250,216.250,12236.00,6170.00,109.56250,125.000,216.250,정상,정상,주의
3,3,17781200.00,77056.0,39520.0,59.863000,244.854000,118.083000,205.333000,23.1250,25796.00,12244.00,243.18800,117.500,117.500,26268.00,13572.00,249.18800,118.750,118.750,24992.00,13704.00,242.18800,118.000,118.000,정상,정상,경고
4,4,10143988.00,0.0,0.0,59.798140,0.000000,133.750000,231.500000,26.8750,0.00,0.00,0.00000,133.750,232.750,0.00,0.00,0.00000,134.500,231.500,0.00,0.00,0.00000,133.000,230.250,경고,정상,정상
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2420560,2420561,6229905.00,25925.5,22606.0,57.945000,52.833000,215.327000,372.958000,19.0630,8631.50,6670.50,50.50000,214.227,214.227,9475.50,8068.50,56.87500,216.628,216.628,7818.50,7867.00,51.12500,215.128,215.128,주의,정상,경고
2420561,2420562,6057307.50,91244.0,47068.0,59.928820,152.156250,224.416672,388.583344,33.7500,30128.00,17328.00,153.53125,225.750,390.750,32064.00,15244.00,157.75000,224.250,386.250,29052.00,14496.00,145.18750,223.250,388.750,정상,정상,주의
2420562,2420563,7966820.00,16822.0,11924.8,59.914000,31.363100,219.233000,379.862000,17.3214,5157.84,3806.66,29.37930,218.862,218.862,5709.16,3945.14,31.91490,219.276,219.276,5954.98,4172.97,32.79530,219.560,219.560,정상,경고,경고
2420563,2420564,0.00,0.0,0.0,0.000000,44.572918,0.000000,0.000000,34.3750,0.00,0.00,47.93750,0.000,0.000,0.00,0.00,43.81250,0.000,0.000,0.00,0.00,41.96875,0.000,0.000,경고,경고,정상


In [ ]:
test

,ID,누적전력량,유효전력평균,무효전력평균,주파수,전류평균,상전압평균,선간전압평균,온도,R상유효전력,R상무효전력,R상전류,R상전압,R상선간전압,S상유효전력,S상무효전력,S상전류,S상전압,S상선간전압,T상유효전력,T상무효전력,T상전류,T상전압,T상선간전압
0,0,5.471566e+06,92992.0,58520.00,59.826096,284.125000,129.416672,224.000000,26.875,30856.0,20032.00,285.3125,129.500,224.250,31472.0,19440.0,286.7500,129.500,224.000,30664.0,19048.0,280.3125,129.250,223.750
1,1,1.814760e-01,0.0,0.00,59.816770,0.000000,229.333328,396.916656,23.125,0.0,0.00,0.0000,230.500,399.250,0.0,0.0,0.0000,229.500,395.000,0.0,0.0,0.0000,228.000,396.500
2,2,1.882077e+07,41165.0,11259.00,59.929000,62.302000,228.333000,395.000000,15.000,13584.0,3798.00,61.6560,228.750,228.750,14114.0,3904.0,63.9690,228.750,228.750,13467.0,3557.0,61.2810,227.500,227.500
3,3,1.425333e+07,72496.0,46050.00,59.826096,131.833328,216.250000,374.166656,28.125,25164.0,14328.00,133.4375,216.000,375.500,24656.0,16654.0,136.1875,217.250,374.500,22676.0,15068.0,125.8750,215.500,372.500
4,4,3.713425e+06,13205.5,21464.75,58.975000,39.125000,215.995000,374.113000,10.000,4327.5,6071.25,34.7500,214.618,214.618,5432.5,7740.5,43.5000,217.289,217.289,3445.5,7653.0,39.1250,216.078,216.078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313262,313263,2.969156e+06,0.0,0.00,59.854070,0.000000,264.083344,457.250000,29.375,0.0,0.00,0.0000,263.500,458.250,0.0,0.0,0.0000,265.250,458.250,0.0,0.0,0.0000,263.500,455.250
313263,313264,8.162921e+05,0.0,0.00,59.854103,0.000000,226.083328,391.166656,28.125,0.0,0.00,0.0000,225.500,390.500,0.0,0.0,0.0000,226.250,392.250,0.0,0.0,0.0000,226.500,390.750
313264,313265,9.445837e+06,0.0,0.00,59.863403,0.000000,130.500000,225.916672,26.875,0.0,0.00,0.0000,130.750,227.500,0.0,0.0,0.0000,131.250,225.750,0.0,0.0,0.0000,129.500,224.500
313265,313266,1.803930e+07,0.0,0.00,59.948000,0.000000,265.667000,460.000000,29.375,0.0,0.00,0.0000,265.500,265.500,0.0,0.0,0.0000,267.000,267.000,0.0,0.0,0.0000,264.500,264.500


In [ ]:
submission

,ID,label_역률평균,label_전류고조파평균,label_전압고조파평균
0,0,정상,정상,정상
1,1,정상,정상,정상
2,2,정상,정상,정상
3,3,정상,정상,정상
4,4,정상,정상,정상
...,...,...,...,...
313262,313263,정상,정상,정상
313263,313264,정상,정상,정상
313264,313265,정상,정상,정상
313265,313266,정상,정상,정상


In [ ]:
train.shape, test.shape, submission.shape

((2420565, 27), (313267, 24), (313267, 4))

In [ ]:
train.columns

Index(['index', '누적전력량', '유효전력평균', '무효전력평균', '주파수', '전류평균', '상전압평균', '선간전압평균',
       '온도', 'R상유효전력', 'R상무효전력', 'R상전류', 'R상전압', 'R상선간전압', 'S상유효전력', 'S상무효전력',
       'S상전류', 'S상전압', 'S상선간전압', 'T상유효전력', 'T상무효전력', 'T상전류', 'T상전압', 'T상선간전압',
       'label_역률평균', 'label_전류고조파평균', 'label_전압고조파평균'],
      dtype='object')

# 데이터 전처리

'온도' 컬럼 소수점 변환
- 사람의 체온을 잴때도 소수점 1자리까지만 재기 때문에 소수점 1자리로 사용
- ex) 35.6도

In [ ]:
train['온도'] = round(train['온도'],1) # 소수점 1자리
test['온도'] = round(test['온도'],1) # 소수점 1자리

In [ ]:
train.columns

Index(['index', '누적전력량', '유효전력평균', '무효전력평균', '주파수', '전류평균', '상전압평균', '선간전압평균',
       '온도', 'R상유효전력', 'R상무효전력', 'R상전류', 'R상전압', 'R상선간전압', 'S상유효전력', 'S상무효전력',
       'S상전류', 'S상전압', 'S상선간전압', 'T상유효전력', 'T상무효전력', 'T상전류', 'T상전압', 'T상선간전압',
       'label_역률평균', 'label_전류고조파평균', 'label_전압고조파평균'],
      dtype='object')

In [ ]:
df_columns = {'누적전력량', '유효전력평균', '무효전력평균', '주파수', '전류평균', '상전압평균', '선간전압평균',
       '온도', 'R상유효전력', 'R상무효전력', 'R상전류', 'R상전압', 'R상선간전압', 'S상유효전력', 'S상무효전력',
       'S상전류', 'S상전압', 'S상선간전압', 'T상유효전력', 'T상무효전력', 'T상전류', 'T상전압', 'T상선간전압'}
       
train['온도'] = train['온도'].fillna(train['온도'].mean())
test['온도'] = test['온도'].fillna(test['온도'].mean())

for col in df_columns:
  train[col] = train[col].fillna(train.groupby(['온도'])[col].transform('mean'))
  test[col] = test[col].fillna(test.groupby(['온도'])[col].transform('mean'))

In [ ]:
for col in df_columns:
  train[col] = train[col].replace(np.nan,0)
  test[col] = test[col].replace(np.nan,0)

In [ ]:
train.isnull().sum()

index            0
누적전력량            0
유효전력평균           0
무효전력평균           0
주파수              0
전류평균             0
상전압평균            0
선간전압평균           0
온도               0
R상유효전력           0
R상무효전력           0
R상전류             0
R상전압             0
R상선간전압           0
S상유효전력           0
S상무효전력           0
S상전류             0
S상전압             0
S상선간전압           0
T상유효전력           0
T상무효전력           0
T상전류             0
T상전압             0
T상선간전압           0
label_역률평균       0
label_전류고조파평균    0
label_전압고조파평균    0
dtype: int64

In [ ]:
test.isnull().sum()

ID        0
누적전력량     0
유효전력평균    0
무효전력평균    0
주파수       0
전류평균      0
상전압평균     0
선간전압평균    0
온도        0
R상유효전력    0
R상무효전력    0
R상전류      0
R상전압      0
R상선간전압    0
S상유효전력    0
S상무효전력    0
S상전류      0
S상전압      0
S상선간전압    0
T상유효전력    0
T상무효전력    0
T상전류      0
T상전압      0
T상선간전압    0
dtype: int64

기술통계
- 온도 별 기술통계
- 주파수 별 기술통계

In [ ]:
from scipy.stats import skew
from scipy.stats import kurtosis

In [ ]:
def range_func(x):
    max_val = np.max(x)
    min_val = np.min(x)
    range_val = max_val - min_val
    return range_val

def iqr_func(x):
    low = 5
    high = 95
    q3, q1 = np.percentile(x, [low, high])
    iqr = q3 - q1
    return iqr

# mad(median_absolute_devation)
def premad(x):
    return np.median(np.absolute(x-np.median(x,axis=0)),axis=0)

# skewness, kurtosis
def preskew(x):
    return skew(x)

def prekurt(x):
    return kurtosis(x,fisher=True)


In [ ]:
# 온도 별 기술통계
#train_preprocess = train[train_not_feature].groupby('온도').agg(['mean','std',range_func,premad,preskew,prekurt])
#test_preprocess = test[test_not_feature].groupby('온도').agg(['mean','std',range_func,premad,preskew,prekurt])
#train_preprocess.columns=[i[0]+'_'+i[1] for i in train_preprocess.columns]
#test_preprocess.columns=[i[0]+'_'+i[1] for i in test_preprocess.columns]

train = pd.concat([train,train_preprocess],axis=1)
test = pd.concat([test,test_preprocess],axis=1)

전력
- 전력(P) = 전압(V)*전류(I)

{'누적전력량', '유효전력평균', '무효전력평균', '주파수', '전류평균', '상전압평균', '선간전압평균',
       '온도', 'R상유효전력', 'R상무효전력', 'R상전류', 'R상전압', 'R상선간전압', 'S상유효전력', 'S상무효전력',
       'S상전류', 'S상전압', 'S상선간전압', 'T상유효전력', 'T상무효전력', 'T상전류', 'T상전압', 'T상선간전압'}

In [ ]:
train['R상전력'] = train['R상전압'] * train['R상전류']
test['R상전력'] = test['R상전압'] * test['R상전류']

train['S상전력'] = train['S상전압'] * train['S상전류']
test['S상전력'] = test['S상전압'] * test['S상전류']

train['T상전력'] = train['T상전압'] * train['T상전류']
test['T상전력'] = test['T상전압'] * test['T상전류']

피상전력
- 피상전력 = 유효전력^2 + 무효전력^2
- 유효전력 = 피상전력 * cos

In [ ]:
#train['피상전력'] = np.sqrt(train['유효전력평균'].astype(float)) + np.sqrt(train['무효전력평균'].astype(float))
#test['피상전력'] = np.sqrt(test['유효전력평균'].astype(float)) + np.sqrt(test['무효전력평균'].astype(float))

역률 (power factor)
- 역률 cos = 유효전력/피상전력

In [ ]:
#train['R상역률cos'] = train['R상유효전력']/train['피상전력']
#test['R상역률cos'] = test['R상유효전력']/test['피상전력']

#train['S상역률cos'] = train['S상유효전력']/train['피상전력']
#test['S상역률cos'] = test['S상유효전력']/test['피상전력']

#train['T상역률cos'] = train['T상유효전력']/train['피상전력']
#test['T상역률cos'] = test['T상유효전력']/test['피상전력']

푸리에 변환
- 적합한 컬럼을 푸리에 변환

In [ ]:
def prepuriae(x):
  fft = np.fft.fft(x) / len(x)
  fft_abs = abs(fft)
  return fft_abs

def prebackpuriae(x):
  fft = np.fft.ifft(x)*len(x)
  fff_abs = abs(fft)
  fff_abs
  return fff_abs

In [ ]:
#train_not_feature = train.columns[(train.columns.str.contains('평균') == False) & (train.columns.str.contains('label_') == False)]
#test_not_feature = test.columns[test.columns.str.contains('평균') == False]
#train['index'].rename('ID')
#train[train_not_feature] = prepuriae(train[train_not_feature])
#test[test_not_feature] = prepuriae(test[test_not_feature])

In [ ]:
train.columns

Index(['index', '누적전력량', '유효전력평균', '무효전력평균', '주파수', '전류평균', '상전압평균', '선간전압평균',
       '온도', 'R상유효전력', 'R상무효전력', 'R상전류', 'R상전압', 'R상선간전압', 'S상유효전력', 'S상무효전력',
       'S상전류', 'S상전압', 'S상선간전압', 'T상유효전력', 'T상무효전력', 'T상전류', 'T상전압', 'T상선간전압',
       'label_역률평균', 'label_전류고조파평균', 'label_전압고조파평균', 'R상전력', 'S상전력', 'T상전력'],
      dtype='object')

In [ ]:
df_columns2 = {'누적전력량', '유효전력평균', '무효전력평균', '주파수', '전류평균', '상전압평균', '선간전압평균',
       '온도', 'R상유효전력', 'R상무효전력', 'R상전류', 'R상전압', 'R상선간전압', 'S상유효전력', 'S상무효전력',
       'S상전류', 'S상전압', 'S상선간전압', 'T상유효전력', 'T상무효전력', 'T상전류', 'T상전압', 'T상선간전압','R상전력', 'S상전력', 'T상전력',
       '피상전력', 'R상역률cos', 'S상역률cos', 'T상역률cos'}

In [ ]:
train.isnull().sum()

index            0
누적전력량            0
유효전력평균           0
무효전력평균           0
주파수              0
전류평균             0
상전압평균            0
선간전압평균           0
온도               0
R상유효전력           0
R상무효전력           0
R상전류             0
R상전압             0
R상선간전압           0
S상유효전력           0
S상무효전력           0
S상전류             0
S상전압             0
S상선간전압           0
T상유효전력           0
T상무효전력           0
T상전류             0
T상전압             0
T상선간전압           0
label_역률평균       0
label_전류고조파평균    0
label_전압고조파평균    0
R상전력             0
S상전력             0
T상전력             0
dtype: int64

In [ ]:
test.isnull().sum()

ID        0
누적전력량     0
유효전력평균    0
무효전력평균    0
주파수       0
전류평균      0
상전압평균     0
선간전압평균    0
온도        0
R상유효전력    0
R상무효전력    0
R상전류      0
R상전압      0
R상선간전압    0
S상유효전력    0
S상무효전력    0
S상전류      0
S상전압      0
S상선간전압    0
T상유효전력    0
T상무효전력    0
T상전류      0
T상전압      0
T상선간전압    0
R상전력      0
S상전력      0
T상전력      0
dtype: int64

In [ ]:
pip install catboost

In [ ]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers
from sklearn.neighbors import KNeighborsClassifier
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression

In [ ]:
train_x = train.drop(['label_역률평균','label_전류고조파평균','label_전압고조파평균'],axis=1)
train_y = train[['label_역률평균','label_전류고조파평균','label_전압고조파평균']]

In [ ]:
(X_train, X_test, y_train, y_test) = train_test_split(train_x, train_y, train_size=0.8, random_state=11)

In [ ]:
X_train.shape, y_train.shape,X_test.shape,y_test.shape

((1936452, 27), (1936452, 3), (484113, 27), (484113, 3))

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train,y_train)

clf = LogisticRegression(solver='newton-cg', max_iter=100)
clf.fit(X_train,y_train)

In [ ]:
y_pred = knn.predict(test)
y_pred1 = clf.predict(test)

In [ ]:
y_pred

array([['정상', '주의', '경고'],
       ['경고', '정상', '경고'],
       ['주의', '정상', '정상'],
       ...,
       ['경고', '정상', '정상'],
       ['정상', '주의', '주의'],
       ['정상', '주의', '정상']], dtype=object)

In [ ]:
y_pred1

In [ ]:
#submission[['label_역률평균','label_전류고조파평균','label_전압고조파평균']] = y_pred
submission[['label_역률평균','label_전류고조파평균','label_전압고조파평균']] = y_pred1

In [ ]:
submission.to_csv('baseline_logistic.csv',index=False)

In [ ]:
submission

,ID,label_역률평균,label_전류고조파평균,label_전압고조파평균
0,0,정상,주의,경고
1,1,경고,정상,경고
2,2,주의,정상,정상
3,3,정상,정상,정상
4,4,경고,주의,경고
...,...,...,...,...
313262,313263,경고,주의,정상
313263,313264,경고,정상,정상
313264,313265,경고,정상,정상
313265,313266,정상,주의,주의


In [ ]:
"""
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
numeric_features = [0,1,2]
numeric_transformer = Pipeline(steps=[('imputer', IterativeImputer(random_state=11)),('scaler',StandardScaler())])
preprocessor = ColumnTransformer(transformers=[('num',numeric_transformer, numeric_features)])
model = Pipeline(steps=[('preprocessor',preprocessor),('classifier',CatBoostClassifier(loss_function='MultiClass',class_weights=[.21,.82,.97],iterations=3000,depth=10,random_state=11))])
model.fit(X_train,y_train)
"""

"\nfrom sklearn.compose import ColumnTransformer\nfrom sklearn.pipeline import Pipeline\nfrom sklearn.impute import SimpleImputer\nfrom sklearn.preprocessing import StandardScaler, OneHotEncoder\nfrom sklearn.experimental import enable_iterative_imputer\nfrom sklearn.impute import IterativeImputer\nnumeric_features = [0,1,2]\nnumeric_transformer = Pipeline(steps=[('imputer', IterativeImputer(random_state=11)),('scaler',StandardScaler())])\npreprocessor = ColumnTransformer(transformers=[('num',numeric_transformer, numeric_features)])\nmodel = Pipeline(steps=[('preprocessor',preprocessor),('classifier',CatBoostClassifier(loss_function='MultiClass',class_weights=[.21,.82,.97],iterations=3000,depth=10,random_state=11))])\nmodel.fit(X_train,y_train)\n"

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.datasets import make_multilabel_classification
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline

In [ ]:
from xgboost import XGBClassifier
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

In [ ]:
#num_classes = len(np.unique(y_train))
#y_train_categorical = tf.keras.utils.to_categorical(y_train, num_classes)

ValueError: ignored

In [ ]:
cat_models={}

def cat_kfold(max_depth, learning_rate, random_seed):
    
    folds=StratifiedKFold(n_splits=10, shuffle=True, random_state=55)
    outcomes=[]
    sub=np.zeros((test.shape[0], 3))  
    
    for seed in random_seed:
        for n_fold, (train_index, val_index) in enumerate(folds.split(train_x, train_y)):
            print(f'===================================={n_fold+1}============================================')
            
            X_train, X_val = train_x.iloc[train_index], train_x.iloc[val_index]
            y_train, y_val = train_y.iloc[train_index], train_y.iloc[val_index]

            # early_stopping 50에서 가장 좋은 점수를 내는 learning_rate를 활용
            cat = CatBoostClassifier(n_estimators=3000, max_depth=max_depth, random_seed=seed, learning_rate=learning_rate, bootstrap_type ='Bernoulli')
            cat.fit(X_train, y_train,
                  eval_set=[(X_train, y_train), (X_val, y_val)],
                  early_stopping_rounds=50, cat_features=cat_features,
                  verbose=100)

            cat_models[n_fold] = cat

            # val 데이터 예측
            predictions = cat.predict_proba(X_val)
            # test 데이터 예측
            test_predictions = cat.predict_proba(test)

            # val 데이터 예측 logloss 값 저장
            logloss=log_loss(to_categorical(y_val), predictions)
            outcomes.append(logloss)
            print(f"FOLD {n_fold+1} : logloss:{logloss}")

            # test 데이터 예측 결과 종합
            # 최종 적으로는 kolds 횟수 만큼 나눠서 평균 값을 활용
            sub+=test_predictions

            print(f'================================================================================\n\n')

    # 저장된 val 데이터 예측 logloss 값의 평균 값으로 성능을 비교
    mean_outcome=np.mean(outcomes)
    print("Mean:{}".format(mean_outcome))
    
    return sub/(folds.n_splits * len(random_seed))

In [ ]:
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=199)
    folds = []
    for train_idx, valid_idx in skf.split(X_train,y_train_categorical):
        folds.append((train_idx, valid_idx))

    lgb_models = {}
    bestmodel = []

    from sklearn.metrics import f1_score

    def macro_f1(y_true, y_pred):
        score = f1_score(y_true=y_true, y_pred=y_pred, average='macro')
        return score

    for fold in range(5):
      print(f'===================================={fold + 1}============================================')
      train_idx, valid_idx = folds[fold]
      x_train, x_test, y_train, y_test = train_features.iloc[train_idx].values, \
                                           train_features.iloc[valid_idx].values, \
                                           train[['label_역률평균','label_전류고조파평균','label_전압고조파평균']][train_idx].values, train[['label_역률평균','label_전류고조파평균','label_전압고조파평균']][valid_idx].values
      model = LGBMClassifier(n_estimators=3000, learning_rate=0.03, random_state=19, reg_lambda=0.3)  # , boosting_type='dart')
      model.fit(x_train, y_train,
                  eval_set=[(x_train, y_train), (x_test, y_test)],
                  early_stopping_rounds=500,
                  verbose=100,
                  eval_metric='multi_logloss')
      lgb_models[fold] = model
      bestmodel.append([fold, model.best_score_['valid_1']['multi_logloss']])
      f_imp = pd.DataFrame({'IMPORTANCE': train_features.columns, 'VALUES': model.feature_importances_}).sort_values('VALUES', ascending=False)
      print(f_imp)
      print(f'================================================================================\n\n')


ValueError: ignored

In [ ]:
pred = xgb_clf.predict(test)

XGBoostError: ignored

In [ ]:
from sklearn import metrics
metrics.f1_score(y_test, pred, average='macro')

In [ ]:
pred

In [ ]:
submission[['label_역률평균','label_전류고조파평균','label_전압고조파평균']] = pred
submission.to_csv('baseline_xgboost.csv',index=False)